In [1]:
import pandas as pd
import numpy as np
import random
import os
import gc
from tqdm import tqdm

# Transformers
import torch
from transformers import AutoTokenizer, AutoModel

C:\Users\kist0\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
# 시드 고정

def seed_everything(seed: int=42):
  random.seed(seed)
  np.random.seed(seed)
  os.environ['PYTHONASHSEED'] = str(seed)

my_seed = 42
seed_everything(my_seed)

# 파일 경로 설정
review_path = "balanced_lyrics.csv"

In [ ]:
#BERT 토크나이저 및 모델 불러오기
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModel.from_pretrained("bert-base-uncased")

In [ ]:
def Tokenize(text_list, tokenizer, model, batch_size: int):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    model = model.to(device).eval() 
    embeddings = []

    for i in tqdm(range(0, len(text_list), batch_size)): # 배치사이즈는 꼭 팀원 모두 동일하게 진행할것
        batch_texts = text_list[i:i + batch_size] 

        inputs = tokenizer(
            batch_texts,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=512
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            cls_embeddings = outputs.last_hidden_state[:, 0, :]
            embeddings.append(cls_embeddings.cpu())

        torch.cuda.empty_cache()

    all_embeddings = torch.cat(embeddings, dim=0).numpy().tolist()
    del inputs, outputs, cls_embeddings
    gc.collect()
    torch.cuda.empty_cache()

    return all_embeddings

In [6]:
def bert(df, column_name='text', batch_size=1):

    print("BERT Embedding Start")
    bert_emb = Tokenize(df[column_name].tolist(), bert_tokenizer, bert_model, batch_size)

    df['bert_emb'] = bert_emb
    return df

In [17]:
balance_df = pd.read_csv(review_path , index_col=0)
print(balance_df.shape)
balance_df.head()

(104098, 18)


,track_uri,track_name,artist,user_id,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,lyrics,like
0,0qCQg5TkfBfkTsQP3IhAmC,Southside Of Heaven,Ryan Bingham,95797a27c7ec61a04f0c9437448eee7233e8a456,379160,0.387,0.499,7,-10.207,1,0.0259,0.002900,0.003730,0.0695,0.333,111.032,"When I die, Lord, oh, won't you put my soul up...",0
1,5B3UjDMiCwWFgnXqt0isd5,Martin Sheen Or JFK,Yellowcard,2e91726a1f802c680a8499544edc3561ce50b965,226746,0.256,0.756,2,-5.395,1,0.0390,0.000128,0.000000,0.1160,0.268,197.512,Here it is\n One more glass for these broken h...,0
2,23qnota5Iyg4f9LNdNIC1D,Televators,The Mars Volta,aafd6caf4b6dad692dd93623cc5c4e5b5a919600,378786,0.306,0.541,11,-8.195,0,0.0383,0.457000,0.000006,0.1220,0.183,122.041,Just as he hit\n The ground\n They lowered a t...,0
3,2sy0icOIskeP2lCqgZiTyE,Talk Show Host,Radiohead,28b78e314a00f86e24ae7ab1e40392861fbdc5a1,281000,0.535,0.479,5,-14.112,0,0.0311,0.277000,0.028500,0.1390,0.504,88.841,I want to\n I want to be someone else or I'll ...,0
4,19olraJp56OeGTQSZwLZc1,Dying Inside,Saint Vitus,8c5d4fe93b5eebbbbe5b80fdac548920c08c395c,445126,0.313,0.406,1,-14.754,1,0.0347,0.027800,0.000437,0.1250,0.218,112.856,I have got to change my ways\n 'Cause I'm losi...,0


In [10]:
# Str로 타입 변경
balance_df['lyrics'] = balance_df['lyrics'].astype("str")

In [12]:
# 컴퓨터 성능에 따라, 파트 분담에 따라 조절 
# 0:5000, 5000:10000 이런 식 (0~4999 5천개, 5000~9999 5천개 ...) 
start = 0
end = 104098
chunk_size = 5000
num = 1

while start < end:
    chunk_end = min(start + chunk_size, end)
    name = f"df{num}"
    
    print(f"\n==== {name} 임베딩 시작 ({start} ~ {chunk_end}) ====")
    sub_df = balance_df[start:chunk_end].copy()
    
    sub_df = bert(sub_df, column_name='lyrics', batch_size=1)

    save_path = f"./lyrics_embeds/lyrics_emb_{name}.csv"  # 저장 경로 설정
    sub_df.to_csv(save_path)
    
    print(f"{save_path} 저장 완료")
    
    start += chunk_size
    num += 1


==== df1 임베딩 시작 (0 ~ 5000) ====
BERT Embedding Start


100%|██████████| 5000/5000 [12:21<00:00,  6.75it/s]


./lyrics_embeds/lyrics_emb_df1.csv 저장 완료

==== df2 임베딩 시작 (5000 ~ 10000) ====
BERT Embedding Start


100%|██████████| 5000/5000 [12:44<00:00,  6.54it/s]


./lyrics_embeds/lyrics_emb_df2.csv 저장 완료

==== df3 임베딩 시작 (10000 ~ 15000) ====
BERT Embedding Start


...

./lyrics_embeds/lyrics_emb_df19.csv 저장 완료

==== df20 임베딩 시작 (95000 ~ 100000) ====
BERT Embedding Start


100%|██████████| 5000/5000 [10:15<00:00,  8.12it/s]


./lyrics_embeds/lyrics_emb_df20.csv 저장 완료

==== df21 임베딩 시작 (100000 ~ 104098) ====
BERT Embedding Start


100%|██████████| 4098/4098 [08:03<00:00,  8.47it/s]


./lyrics_embeds/lyrics_emb_df21.csv 저장 완료


In [7]:
# 만든 csv 파일들 합치기

all = []
for i in range(1, 22):
    df_part = pd.read_csv(f"./lyrics_embeds/lyrics_emb_df{i}.csv", index_col=0)
    all.append(df_part)

df_full = pd.concat(all).reset_index(drop=True)

In [ ]:
# 합친파일 내보내기

save_path = f"./lyrics_embeds/emb_all_.csv"
df_full.to_csv(save_path)
print(f"{save_path} 저장 완료")

./lyrics_embeds/emb_all.csv 저장 완료
